In [4]:
import os
from tensorflow.keras import models, layers, activations, optimizers, utils, losses, initializers, metrics, callbacks
import numpy as np 
import cv2 as cv 
import matplotlib as plt 

In [5]:
epochs = 100
batch_size = 32
patience = 25
learning_rate = 0.001
model_path = 'checkpoints/model.keras'

In [11]:
exists = os.path.exists(model_path)

model = models.load_model(model_path) \
    if exists \
        else models.Sequential([
            layers.Resizing(30, 30),
            layers.Rescaling(scale=1./127.5, offset=-1),
            layers.RandomRotation((-0.2, 0.2)),
            layers.Conv2D(64, (3, 3),
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(64, (3, 3),
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.MaxPooling2D((2, 2)),
            layers.BatchNormalization(),
            layers.Flatten(),
            layers.Dropout(0.5),
            layers.Dense(128,
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.Dropout(0.5),
            layers.Dense(256,
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.Dropout(0.5),
            layers.Dense(512,
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.Dropout(0.5),
            layers.Dense(256,
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.Dense(62,
                activation = 'sigmoid',
                kernel_initializer = initializers.RandomNormal()
            )
        ])

if exists:
    model.summary()
else:
    model.compile(
        optimizer = optimizers.Adam(
        learning_rate = learning_rate
        ),
        loss = losses.SparseCategoricalCrossentropy(),
        metrics = [ 'accuracy' ]
    )

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         (None, 30, 30, 3)         0         
                                                                 
 rescaling (Rescaling)       (None, 30, 30, 3)         0         
                                                                 
 random_rotation (RandomRot  (None, 30, 30, 3)         0         
 ation)                                                          
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 64)        1792      
                                                                 
 random_flip (RandomFlip)    (None, 28, 28, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 64)        0         
 D)                                                     

In [12]:
train = utils.image_dataset_from_directory(
    "Img", # Change path
    validation_split= 0.2,
    subset= "training",
    seed= 123,
    shuffle= True,
    image_size= (128, 128),
    batch_size= batch_size
)

test = utils.image_dataset_from_directory(
    "Img", # Change path
    validation_split= 0.2,
    subset= "validation",
    seed= 123,
    shuffle= True,
    image_size= (128, 128),
    batch_size= batch_size
)

Found 3410 files belonging to 62 classes.
Using 2728 files for training.
Found 3410 files belonging to 62 classes.
Using 682 files for validation.


In [13]:
model.fit(train,
    epochs = epochs,
    validation_data = test,
    callbacks= [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            verbose = 1
        ),
        callbacks.ModelCheckpoint(
            filepath = model_path,
            save_weights_only = False,
            monitor = 'loss',
            mode = 'min',
            save_best_only = True
        )
    ]
)

Epoch 1/100


86/86 [==============================] - 5s 36ms/step - loss: 3.1916 - accuracy: 0.1239 - val_loss: 3.0740 - val_accuracy: 0.1173
Epoch 2/100
86/86 [==============================] - 3s 35ms/step - loss: 3.1226 - accuracy: 0.1375 - val_loss: 3.7200 - val_accuracy: 0.0865
Epoch 3/100
86/86 [==============================] - 3s 35ms/step - loss: 3.1631 - accuracy: 0.1367 - val_loss: 3.4372 - val_accuracy: 0.0997
Epoch 4/100
86/86 [==============================] - 3s 36ms/step - loss: 3.1160 - accuracy: 0.1327 - val_loss: 3.2175 - val_accuracy: 0.1261
Epoch 5/100
86/86 [==============================] - 3s 33ms/step - loss: 3.1273 - accuracy: 0.1272 - val_loss: 3.1100 - val_accuracy: 0.1217
Epoch 6/100
86/86 [==============================] - 3s 38ms/step - loss: 3.1444 - accuracy: 0.1331 - val_loss: 3.0213 - val_accuracy: 0.1540
Epoch 7/100
86/86 [==============================] - 3s 33ms/step - loss: 3.1313 - accuracy: 0.1397 - val_loss: 3.2341 - val_accuracy: 0.1026
Epoch 8/100
86/86 